#### Make sure to install the following packages

In [ ]:
!pip install urllib3

In [ ]:
!pip install chardet

In [ ]:
!pip install kafka-python

#### Code implementing stream: Source -> Kafka

In [ ]:
import requests
import json
from kafka import KafkaProducer
import time

# Kafka configuration
KAFKA_BROKER = 'localhost:9092'
TOPIC = 'weatherlogs'

# Create Kafka Producer
producer = KafkaProducer(
    bootstrap_servers=[KAFKA_BROKER],
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Define latitude and longitude for the desired location
latitude = 38.8977  # Example: Washington, D.C.
longitude = -77.0365

def stream_forecast():
    while True:
        # Fetch forecast data
        api_url = f"https://api.weather.gov/points/{latitude},{longitude}"
        response = requests.get(api_url)
        
        if response.status_code == 200:
            data = response.json()
            forecast_url = data['properties']['forecast']

            forecast_response = requests.get(forecast_url)
            if forecast_response.status_code == 200:
                forecast_data = forecast_response.json()
                for period in forecast_data['properties']['periods']:
                    # Create a data dictionary for Kafka
                    forecast_info = {
                        "location": "Washington, D.C.",
                        "short_forecast": period['shortForecast'],
                        "temperature": period['temperature'],
                        "temperature_unit": period['temperatureUnit'],
                        "wind_speed": period['windSpeed'],
                        "start_time": period['startTime']
                    }

                    # Send data to Kafka
                    producer.send(TOPIC, value=forecast_info)
                    print(f"Data streamed to Kafka: {forecast_info}")
                
                # Wait before fetching again
                time.sleep(604800)  # Stream every 7 days
            else:
                print("Failed to fetch forecast data.")
        else:
            print("Failed to fetch location data.")

try:
    stream_forecast()
except KeyboardInterrupt:
    print("Streaming stopped.")
finally:
    producer.close()
